<a href="https://colab.research.google.com/github/pitapatat/wanted_pre_onboarding/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C_%ED%94%84%EB%A6%AC%EC%98%A8%EB%B3%B4%EB%94%A9_%EC%BD%94%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (1) Tokenizer 생성하기

In [ ]:
# library
import re

In [ ]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

    # 텍스트 전처리 함수
    def preprocessing(self, sequences):

        ''' - input(sequences)
            - 정규표현식 통해 소문자 변환, 특수문자 제거
            - output(result): whitespace 단위로 토큰화된 단어(nested list)
            '''

        result = []
        for word in sequences:
            tokenized= re.sub(r'[^a-z0-9 ]', '', word.strip().lower()).split(' ')
            result.append(tokenized)
        print(f"tokenized sentences : {result}")
        return result   
    
    # 어휘사전 구축 함수
    def fit(self, sequences):

        ''' - 텍스트 전처리 함수 수행 후 반환값 : 토큰화된 단어(tokens)
            - 중복 제거 후 토큰화된 단어에 정수 인덱싱 
            - 어휘사전(word_dict) 생성(dictionary)
            '''

        self.fit_checker = False
        total_words = []
        tokens = self.preprocessing(sequences)
        for token in tokens:
            total_words.extend(token)
        words = list(set(total_words))

        for i, word in enumerate(words):
            self.word_dict[word] = i+1
        print(f"word_dict: {self.word_dict}, 어휘사전 길이: {len(self.word_dict)}")
        self.fit_checker = True
    
    # 입력 문장 정수 인덱싱 함수
    def transform(self, sequences):

        ''' - 어휘사전 활용하여 문자 -> 정수 인덱싱
            - 사전에 없는 단어는 0으로 인덱싱
            - output(result): 입력문장 정수 인덱싱 결과(nested list)
            '''

        result = []
        tokens = self.preprocessing(sequences)    
        if self.fit_checker:
            for token in tokens:
                indexes = [self.word_dict[i] if i in list(self.word_dict.keys()) else self.word_dict['oov'] for i in token]
                result.append(indexes)  
            #print(f"result of indexing for each sentence : {result}")
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
    
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result


## (2) TfidfVectorizer 생성하기


In [ ]:
#library
import numpy as np
from collections import Counter

In [ ]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
    
    # idf 행렬 생성 함수
    def fit(self, sequences):

        ''' - input(sequences): Tokenizer() 통해 토큰화 -> 정수 인덱싱 
            - n:전체 문장수(len(sequences)), d:문장(d = 1,..,len(sequences)-1), t: 어휘사전 내 토큰화된 단어(t = 1,.., len(self.word_dict)))
            - df(d,t): 단어 t가 포함된 문장 d의 갯수 
            - idf(d,t) = np.log(n/1+df(d,t))
            '''

        tokenized = self.tokenizer.fit_transform(sequences)

        df_dict = Counter()  # 특정 단어(index)가 포함된 문장 갯수 세기(df_dict)
        for token in tokenized:
            df_dict.update(token)    
        #print(f"df_ditc: {df_dict}") 
        
        # 어휘사전의 단어(word_dict)를 기준으로 특정 단어가 포함된 문장의 갯수를 1차원 vector로 표현
        df_matrix = [df_dict[i] if i in df_dict.keys() else 0 for i in range(len(self.tokenizer.word_dict))]
        print(f"df_matrix: {df_matrix}") 

        n = len(tokenized)  # 전체 입력문장의 수    
        idf_matrix = [np.round(np.log(n/(df+1)), 6) for df in df_matrix]  # 공식에 따른 idf_matrix 생성
        idf_matrix = [0 if idf < 0 else idf for idf in idf_matrix]   # idf < 0 (문서 갯수보다 많이 등장)인 경우: 0 으로 치환 
        print(f"idf_matrix: {idf_matrix}") 
        self.idf_matrix = idf_matrix
        #assert len(idf_matrix) == len(self.tokenizer.word_dict)    
        self.fit_checker = True
        
    # tf-idf 행렬 생성 함수
    def transform(self, sequences):

        ''' - tf(d,t): 문장 d에 단어 t가 등장한 횟수
            - tf(d,t)*idf(d,t)  
            ''' 

        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)

            tf_matrix = []   # 어휘사전의 단어를 기준으로 각 문장마다 단어가 등장한 횟수(nested list)      
            for token in tokenized:
                tf = [1 if i in token else 0 for i in range(len(self.tokenizer.word_dict))]
                tf_matrix.append(tf)
            #print(f"tf_matrix: {tf_matrix}, len(tf_matrix): {len(tf_matrix)}")
            
            tfidf_matrix = []   # tf*idf(fit 함수를 통한 결과(idf_matrix)활용)
            for tf in tf_matrix:
                tf_idf = [i*j for i , j in zip(tf,self.idf_matrix)]
                tfidf_matrix.append(tf_idf)            
            #print(f"tf-idf matrix: {tfidf_matrix}")
            self.tfidf_matrix = tfidf_matrix
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")
   
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)
